In [10]:
%env SPARK_VERSION=3.3 # in cmd >pyspark --version  - to check version  # here used 3.3 becouse pydeequ works with =<3.3

env: SPARK_VERSION=3.3 # in cmd >pyspark --version  - to check version  # here used 3.3 becouse pydeequ works with =<3.3


In [2]:
import pydeequ
from pyspark.sql import SparkSession, Row

In [3]:
server_url = "jdbc:sqlserver://host.docker.internal" 
database_name = "TRN"
url = server_url + ";" + "databaseName=" + database_name + ";trustServerCertificate=True;"

table = "hr.employees"
user = "oleksandraLogin" # your user creds here
password  = "StrongPassword1@" # your user creds here

In [4]:
# TODO connect to DB with Spark using JDBC connection to read the data

connection_details = { "user": user, "password": password, "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver", }

spark = (SparkSession
    .builder
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .config("spark.driver.extraClassPath", "/home/jovyan/mssql-jdbc-12.6.1.jre11.jar")
    .getOrCreate())


#mssql-jdbc-12.6.1.jre11.jar were uploaded from
# https://learn.microsoft.com/en-us/sql/connect/jdbc/download-microsoft-jdbc-driver-for-sql-server?view=sql-server-ver16#download
# -> unzip 
# -> copied into podmman container by using import file from jupiter notebook
# -> comand >podman cp mssql-jdbc-12.6.1.jre11.jar eb28ed05474b:/home/jovyan/work
# or comand  >podman machine ssh sh -c 'cat > //home/jovyan/work/ mssql-jdbc-12.6.1.jre11.jar < mssql-jdbc-12.6.1.jre11.jar 
# doesn't work. As reason: I use Windows cmd. Podman works with Linuks
# for future try to use Linux VM

In [5]:
df = spark.read.jdbc(url=url, table=table, properties=connection_details)

In [6]:
df.printSchema()  # test comand

root
 |-- employee_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- hire_date: date (nullable = true)
 |-- job_id: integer (nullable = true)
 |-- salary: decimal(8,2) (nullable = true)
 |-- manager_id: integer (nullable = true)
 |-- department_id: integer (nullable = true)



In [7]:
df.show(10,False)   # test comand

+-----------+----------+---------+--------------------------------+------------+----------+------+--------+----------+-------------+
|employee_id|first_name|last_name|email                           |phone_number|hire_date |job_id|salary  |manager_id|department_id|
+-----------+----------+---------+--------------------------------+------------+----------+------+--------+----------+-------------+
|100        |Steven    |King     |steven.king@sqltutorial.org     |515.123.4567|1987-06-17|4     |24000.00|NULL      |9            |
|101        |Neena     |Kochhar  |neena.kochhar@sqltutorial.org   |515.123.4568|1989-09-21|5     |17000.00|100       |9            |
|102        |Lex       |De Haan  |lex.de haan@sqltutorial.org     |515.123.4569|1993-01-13|5     |17000.00|100       |9            |
|103        |Alexander |Hunold   |alexander.hunold@sqltutorial.org|590.423.4567|1990-01-03|9     |9000.00 |102       |6            |
|104        |Bruce     |Ernst    |bruce.ernst@sqltutorial.org     |59

In [21]:
### Data Analyzers section

from pydeequ.analyzers import *
analysisResult = AnalysisRunner(spark) \
                    .onData(df) \
                    .addAnalyzer(Completeness("employee_id")) \
                    .addAnalyzer(Completeness("email")) \
                    .addAnalyzer(Completeness("phone_number")) \
                    .addAnalyzer(CountDistinct("employee_id")) \
                    .addAnalyzer(Size()) \
                    .addAnalyzer(Mean("salary")) \
                    .addAnalyzer(Maximum("salary")) \
                    .addAnalyzer(Minimum("salary")) \
                    .addAnalyzer(Compliance("salary", "salary > 0")) \
                    .run()
                    
analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult)
analysisResult_df.show()

+-------+------------+-------------+-------+
| entity|    instance|         name|  value|
+-------+------------+-------------+-------+
| Column|      salary|   Compliance|    1.0|
| Column| employee_id| Completeness|    1.0|
| Column|       email| Completeness|    1.0|
| Column| employee_id|CountDistinct|   40.0|
|Dataset|           *|         Size|   40.0|
| Column|phone_number| Completeness|   0.85|
| Column|      salary|         Mean| 8060.0|
| Column|      salary|      Maximum|24000.0|
| Column|      salary|      Minimum| 2500.0|
+-------+------------+-------------+-------+



In [24]:
### Data profiling section

from pydeequ.profiles import *

result = ColumnProfilerRunner(spark) \
     .onData(df) \
     .run()

# for col, profile in result.profiles.items():
#     print(profile)

Py4JJavaError: An error occurred while calling o169.run.
: com.amazon.deequ.analyzers.runners.MetricCalculationRuntimeException: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 47.0 failed 1 times, most recent failure: Lost task 0.0 in stage 47.0 (TID 29) (daa1d16ee388 executor driver): java.lang.NoSuchMethodError: 'org.apache.spark.sql.types.StructType org.apache.spark.sql.types.StructType$.fromAttributes(scala.collection.Seq)'
	at org.apache.spark.sql.catalyst.expressions.aggregate.StatefulHyperloglogPlus.aggBufferSchema(StatefulHyperloglogPlus.scala:62)
	at org.apache.spark.sql.execution.aggregate.AggregationIterator.$anonfun$initializeAggregateFunctions$1(AggregationIterator.scala:110)
	at org.apache.spark.sql.execution.aggregate.AggregationIterator.$anonfun$initializeAggregateFunctions$1$adapted(AggregationIterator.scala:81)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at org.apache.spark.sql.execution.aggregate.AggregationIterato